In [1]:

import time as tm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import re

# Set Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless=new')   # Running Chrome in headless mode
chrome_options.add_argument('--no-sandbox')    # Necessary for running in some environments

# Automatically download and use the correct version of chromedriver
service = Service(ChromeDriverManager().install())

# Initialize the WebDriver with the service and options
driver = webdriver.Chrome(service=service, options=chrome_options)
# Load the data
data_dict_check = pd.read_csv("phones_dict_check.csv")
data_raw = pd.read_csv('split_1.csv')

In [2]:
def find_hardware(driver):
    elements = {
        'release_date': '//span[@data-spec="released-hl"]',
        'screen_size': '//span[@data-spec="displaysize-hl"]',
        'battery_capacity': '//span[@data-spec="batsize-hl"]',
        'operating_system': '//span[@data-spec="os-hl"]',
        'chipset': '//div[@data-spec="chipset-hl"]',
        'water_resistance': '//td[@data-spec="bodyother"]',
        'cam1modules': '//td[@data-spec="cam1modules"]',
        'cam2modules': '//td[@data-spec="cam2modules"]',
        'screen_resolution': '//div[@data-spec="displayres-hl"]',
        'videopixels': '//div[@data-spec="videopixels-hl"]',
        'nettech': '//a[@data-spec="nettech"]'
    }

    hardware_info = {}
    
    for key, xpath in elements.items():
        try:
            hardware_info[key] = driver.find_element(By.XPATH, xpath).text.strip()
        except:
            hardware_info[key] = None
    
    return hardware_info

In [3]:
def add_row_to_df(df, result, result1):
    # Kiểm tra để đảm bảo result có đủ thông tin
    if len(result) < 6:
        raise ValueError("result must contain at least 6 elements")
    
    # Tạo dictionary theo đúng thứ tự của các cột
    result_dict = {
        'branch': result[0],               # Thương hiệu
        'name_product': result[1],          # Tên sản phẩm
        'launch_date': None,                # Ngày đăng bán
        'release_date': None,               # Ngày ra mắt
        'old_price': result[2],             # Giá cũ
        'new_price': result[3],             # Giá mới
        'condition': result[4],             # Tình trạng
        'ram': result[6],                        # RAM
        'rom': result[7],                        # ROM
        'battery_capacity': None,           # Dung lượng pin
        'operating_system': None,           # Hệ điều hành
        'chipset': None,                    # Chipset
        'screen_size': None,                # Kích thước màn hình
        'screen_resolution': None,          # Độ phân giải màn hình
        'videopixels': None,                # Độ phân giải video
        'num_cameras': None,                # Số lượng camera
        'wide_camera': None,                # Camera wide (chính)
        'ultrawide_camera': None,           # Camera ultrawide
        'macro_camera': None,               # Camera macro
        'telephoto_camera': None,           # Camera telephoto
        'cam1modules': None,                # Camera chính (số module)
        'cam2modules': None,                # Camera phụ (số module)
        'water_resistance': None,           # Khả năng chống nước
        'nettech': None,                    # Công nghệ mạng
        'product_link': result[5]           # Liên kết chi tiết sản phẩm
    }
    for i in result1:
        if i in result_dict:
            result_dict[i] = result1[i]
    # Tạo DataFrame từ dictionary
    new_row = pd.DataFrame([result_dict], columns=df.columns)
    
    # Thêm hàng mới vào DataFrame
    df = pd.concat([df, new_row], ignore_index=True)
    
    return df

In [4]:
def extract_camera_info(description):
    # Chỉ lưu trữ thông tin MP của từng loại camera
    camera_info = {
        'wide_camera': None,
        'ultrawide_camera': None,
        'macro_camera': None,
        'telephoto_camera': None
    }

    
    cameras = description.split('\n')

    total_cameras = 0 

    
    telephoto_mps = []

    for camera in cameras:
        
        if ('wide' in camera) and ('ultrawide' not in camera):
            mp = re.search(r'(\d+)\sMP', camera)
            if mp:
                camera_info['wide_camera'] = mp.group()
                total_cameras += 1
        
        if 'ultrawide' in camera:
            mp = re.search(r'(\d+)\sMP', camera)
            if mp:
                camera_info['ultrawide_camera'] = mp.group()
                total_cameras += 1
        
        if 'macro' in camera:
            mp = re.search(r'(\d+)\sMP', camera)
            if mp:
                camera_info['macro_camera'] = mp.group()
                total_cameras += 1
        
        if 'telephoto' in camera:
            mp = re.search(r'(\d+)\sMP', camera)
            if mp:
                telephoto_mps.append(int(mp.group(1)))  

    if telephoto_mps:
        camera_info['telephoto_camera'] = f"{max(telephoto_mps)} MP"
        total_cameras += len(telephoto_mps)


    return camera_info, total_cameras


In [5]:
def data_processing1(hardware_info):
    result = {}

    for detail in hardware_info:
        if detail == 'operating_system' and hardware_info[detail]:
            result['operating_system'] = hardware_info[detail].split(",")[0].strip()
        
        elif detail == 'cam1modules' and hardware_info[detail]:
            camera_info, total_cameras = extract_camera_info(hardware_info[detail])
            result['num_cameras'] = total_cameras
            result['wide_camera'] = camera_info['wide_camera']
            result['ultrawide_camera'] = camera_info['ultrawide_camera']
            result['macro_camera'] = camera_info['macro_camera']
            result['telephoto_camera'] = camera_info['telephoto_camera']
            result['cam1modules'] = camera_info['wide_camera']
            
        elif detail == 'cam2modules' and hardware_info[detail]:
            match = re.search(r'(\d+)\s*MP', hardware_info[detail])
            if match:
                result['cam2modules'] = match.group(1) + "MP"
        
        elif detail == 'nettech' and hardware_info[detail]:
            result['nettech'] = hardware_info[detail].split('/')[-1].strip()
        
        elif detail == 'water_resistance' and hardware_info[detail]:
            result['water_resistance'] = 'Yes'
        elif detail == 'release_date' and hardware_info[detail]:
            if 'Released' in hardware_info[detail]:
                done = hardware_info[detail]
                done = done.replace("Released", "")
                done = done.strip()
                result[detail] = done
            else:
                result[detail] = hardware_info[detail]
        if detail not in result:
            result[detail] = hardware_info[detail]  
    
    return result

In [6]:
def find_max_ram(storage_string, target_rom):
    items = [item.strip() for item in storage_string.split(',')]
    max_ram = 0
    for item in items:
        if target_rom in item: 
            # Tách ROM và RAM
            parts = item.split()  
            rom, ram = parts[0], int(parts[1].replace('GB', ''))  # Lấy giá trị RAM là số nguyên
            if ram > max_ram:  # So sánh RAM lớn nhất
                max_ram = ram
    
    return f"{max_ram} GB" if max_ram > 0 else None

In [7]:
def data_processing(index, index1,driver):
    row_data_raw = data_raw.loc[index1]
    branch = data_dict_check.loc[index]['Brand']
    name_product = branch + ' ' + data_dict_check.loc[index]['Phone Name']
    old_price = row_data_raw['oldprice']
    new_price = row_data_raw['newprice']
    condition = row_data_raw['combined_condition']

    rom = row_data_raw['rom'].strip()
    ram = row_data_raw['ram']
    if pd.isna(ram) or ram == None:  # Kiểm tra NaN
        print("RAM is NaN, fetching from web")
        try:
            check = rom.split(" ")[0]  # Giả định rằng ROM chứa dung lượng (VD: '256 GB')
            ram_find = driver.find_element(By.XPATH, '//td[@data-spec="internalmemory"]').text.strip()
            ram = find_max_ram(ram_find, check)
            print('Fetched RAM successfully:', ram)
        except Exception as e:
            print('Error finding RAM:', e)
            ram = None  # Nếu không thể lấy RAM, gán giá trị None
    else:
        if 'Mở rộng' in ram:
            ram = ram.split('+')[0].strip()
        print(f'RAM from data: {ram}')

    result = [branch, name_product, old_price, new_price, condition, data_dict_check.loc[index]['Link'],ram, rom ]
    return result

In [ ]:

columns = [
    # Thông tin chung
    'branch',               # Thương hiệu
    'name_product',         # Tên sản phẩm
    'launch_date',          # Ngày đăng bán
    'release_date',         # Ngày ra mắt

    # Giá cả và tình trạng
    'old_price',            # Giá cũ
    'new_price',            # Giá mới
    'condition',            # Tình trạng

    # Thông số phần cứng
    'ram',                  # RAM
    'rom',                  # ROM
    'battery_capacity',     # Dung lượng pin
    'operating_system',     # Hệ điều hành
    'chipset',              # Chipset

    # Màn hình và camera
    'screen_size',          # Kích thước màn hình
    'screen_resolution',    # Độ phân giải màn hình
    'videopixels',          # Độ phân giải video
    'num_cameras',          # Số lượng camera
    'wide_camera',          # Camera wide (chính)
    'ultrawide_camera',     # Camera ultrawide
    'macro_camera',         # Camera macro
    'telephoto_camera',     # Camera telephoto
    'cam1modules',          # Camera chính (số module)
    'cam2modules',          # Camera phụ (số module)

    # Khả năng và công nghệ
    'water_resistance',     # Khả năng chống nước
    'nettech',              # Công nghệ mạng

    # Thông tin thêm
    'product_link'          # Liên kết chi tiết sản phẩm
]



# Tạo DataFrame rỗng với các cột đã định nghĩa
df_empty = pd.DataFrame(columns=columns)
count = 0
name_ok = {}
name_fail = {}
check = ['Điện Thoại Di Động', 'Điện thoại di động', 'Điện Thoại', 'Điện thoại']


name_replacements = {
    'Apple iPhone SE 2022': 'Apple iPhone SE (2022)',
    'Apple iPhone SE 2020': 'Apple iPhone SE (2020)',
    'Samsung Galaxy A15 LTE': 'Samsung Galaxy A15',
    'Xiaomi Redmi Note 11 Pro PLUS': 'Xiaomi Redmi Note 11 Pro+ 5G',
    'Xiaomi Redmi Note 11 Pro Plus': 'Xiaomi Redmi Note 11 Pro+ 5G',
    'Xiaomi Redmi Note 13 Pro PLUS': 'Xiaomi Redmi Note 13 Pro+',
    'Xiaomi Redmi Note 13 Pro Plus': 'Xiaomi Redmi Note 13 Pro+',
    'Infinix Hot 40 Pro Free Fire': 'Infinix Hot 40 Pro'
}


for index1, name1 in enumerate(data_raw['phone_name'].values):
    print(count)
    try:
        # if count == 5:
        #     break
        name1 = name1.strip()

        if 'iPhone' in name1 and 'Apple' not in name1:
            name1 = 'Apple ' + name1

        name1 = name_replacements.get(name1, name1)
        if 'Plus' in name1 and 'Samsung' in name1:
            name1 = name1.replace('Plus', "").strip() + '+'
        for word in check:
            name1 = name1.replace(word, "").strip()
        matched = False
        for index, name in enumerate(data_dict_check['combined'].values):
            if name1.lower() == name.strip().lower() or name1.lower() + ' 5g' == name.strip().lower():
                driver.get(data_dict_check.loc[index]['Link'])
                tm.sleep(3)
                result = data_processing(index, index1,driver)
                hardware_info = find_hardware(driver)
                result1 = data_processing1(hardware_info)
                df_empty = add_row_to_df(df_empty, result, result1 )
                name_ok[index1] = name1
                count += 1
                matched = True
                break
        if not matched:
            name_fail[index1] = name1
    except Exception as e:
        print(name1)
        print("Error: ", e)


driver.quit()
print(f"Matched count: {count}")
df_empty.to_csv('ADY_data_DONE.csv', index=False, encoding='utf-8')


0
RAM from data: 6 GB
1
RAM from data: 12 GB
2
RAM from data: 8 GB
3
RAM is NaN, fetching from web
Fetched RAM successfully: 8 GB
4
RAM from data: 6 GB
5
RAM from data: 8 GB
6
RAM from data: 6 GB
7
RAM is NaN, fetching from web
Fetched RAM successfully: 6 GB
8
RAM is NaN, fetching from web
Fetched RAM successfully: 6 GB
9
RAM from data: 12 GB
10
RAM from data: 8 GB
11
RAM is NaN, fetching from web
Fetched RAM successfully: 6 GB
12
RAM from data: 8 GB
13
RAM from data: 6 GB
14
RAM from data: 6 GB
15
RAM from data: 8 GB
16
16
RAM from data: 4 GB
17
RAM from data: 6 GB
18
RAM from data: 8 GB
19
RAM from data: 6 GB
20
Apple iPhone 11 Pro
Error:  'float' object has no attribute 'strip'
20
RAM from data: 3 GB
21
RAM from data: 6 GB
22
RAM from data: 4 GB
23
RAM from data: 6 GB
24
RAM is NaN, fetching from web
Fetched RAM successfully: 8 GB
25
RAM from data: 6 GB
26
RAM from data: 4 GB
27
RAM is NaN, fetching from web
Fetched RAM successfully: 3 GB
28
RAM is NaN, fetching from web
Fetched RAM